In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
import seaborn as sns




ecg_dataset=pd.read_csv("/content/ecg.csv")

scaler=StandardScaler()
df=scaler.fit_transform(ecg_dataset)

x_train, x_test=train_test_split(df,test_size=0.2)





encoder=models.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.Dense(32,activation='relu'),
    layers.Dense(16,activation='relu'),
    layers.Dense(8,activation='relu')
])



decoder=models.Sequential([
    layers.Input(shape=(8,)),
    layers.Dense(16,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(x_train.shape[1],activation='linear')
])

autoencoder=models.Sequential([
    encoder,
    decoder
])


autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(x_train,x_train, epochs=10, batch_size=30, shuffle=True)




y_pred=autoencoder.predict(x_test)
mse=np.mean(np.power(x_test-y_pred,2),1)



threshold=np.percentile(mse,95)
anomalies=mse > threshold
num_anomalies=np.sum(anomalies)

print("Number of Anomalies" ,num_anomalies)



# plot of mse vs anomalies threshold

plt.plot(mse,marker='o',linestyle='',markersize=3,label="MSE")
plt.axhline(threshold, linestyle='--', color='r',label="Anomaly threshold")
plt.xlabel('Sample Index')
plt.ylabel('MSE')
plt.legend()
plt.show()


#Original vs Reconstructed for normal data
plt.figure(figsize=(12,6))
plt.plot(x_test[0],label='Original ECG')
plt.plot(y_pred[0],label='Reconstructed ECG')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show


anomalies_index=[index for index, anamoly in enumerate(anomalies) if anamoly]
n=4
anomaly_index=anomalies_index[n]


plt.figure(figsize=(12,6))
plt.plot(x_test[anomaly_index],label='Original ECG')
plt.plot(y_pred[anomaly_index],label='Reconstructed ECG')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show



sns.heatmap(confusion_matrix(anomalies, anomalies), annot=True, fmt='d')
plt.xlabel("predicted label")
plt.ylabel('True label')
plt.title('confusion matrix')
plt.show()
